In [23]:
import pandas as pd

## leer 6 archivos de excel distintos
customerAddress = pd.read_csv("../data/raw/200K_CustomerAddress.csv")
generalOrder= pd.read_csv("../data/raw/200K_GeneralOrderDetail.csv")
individualCustomer = pd.read_csv("../data/raw/200K_IndividualCustomer.csv")
ordersList = pd.read_csv("../data/raw/200K_OrdersList.csv")
productOrderDetail = pd.read_csv("../data/raw/200K_ProductOrderDetail.csv")
productCatalog = pd.read_csv("../data/raw/Product_Catalog.csv")

/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_22662/2328653297.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  customerAddress = pd.read_csv("../data/raw/200K_CustomerAddress.csv")
/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_22662/2328653297.py:5: DtypeWarning: Columns (8,39) have mixed types. Specify dtype option on import or set low_memory=False.
  generalOrder= pd.read_csv("../data/raw/200K_GeneralOrderDetail.csv")
/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_22662/2328653297.py:6: DtypeWarning: Columns (7,8,25) have mixed types. Specify dtype option on import or set low_memory=False.
  individualCustomer = pd.read_csv("../data/raw/200K_IndividualCustomer.csv")
/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_22662/2328653297.py:7: DtypeWarning: Columns (9,12,24) have mixed types. Specify dtype option on import or set low_memory=False.
  ordersList = pd.read_csv("../data/r

# DATA EXPLORATORY ANALYSIS    

In [24]:
customerAddress.head(10)

,addressName,addressType,city,country,countryfake,geoCoordinate,neighborhood,postalCode,state,userId,...,createdIn,updatedBy,updatedIn,lastInteractionBy,lastInteractionIn,followers,tags,auto_filter,Cretaed_Timestamp,Updaqted_Timestamp
0,b7f847462a984eaea560b8b79,residential,GUADALUPE,MEX,NaN,"[-100.2056884765625, 25.717365264892578]",Serena,67129,NUEVO LEÓN,70f350f5-f62e-11ec-835d-0a8fb171123f,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
1,b7999b2bcf154649a42ec1e66,residential,SAN NICOLÁS DE LOS GARZA,MEX,NaN,"[-100.27811644319432, 25.74423676411608]",Las Puentes Sector 1,66460,NUEVO LEÓN,c21ef477-e9c4-11ec-835d-02978ed58bf1,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
2,050e6f3891bf4708aa69bc703,residential,MONTERREY,MEX,NaN,"[-100.26667022705078, 25.69341278076172]",Fontanares Churubusco Sur,64540,NUEVO LEÓN,66021709-f670-11ec-835d-16b245a39a51,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
3,e80996901a5c43cba050796fb,residential,XOCHIMILCO,MEX,NaN,"[-99.16278076171875, 19.291709899902344]",San Lorenzo La Cebada,16035,CIUDAD DE MÉXICO,79e873e1-e2c7-11ec-835d-1205375cb899,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
4,651e1032d4604100a9d409749,residential,GUADALUPE,MEX,NaN,"[-100.24179077148438, 25.6845645904541]",Azteca,67150,NUEVO LEÓN,89975ae6-f33a-11ec-835d-0ef358621195,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
5,ea3e0f5bb23e47efac1b03e46,residential,MONTERREY,MEX,NaN,"[-100.30682373046875, 25.643972396850586]",Independencia,64720,NUEVO LEÓN,74dc8cff-e529-11ec-835d-12a16ce67027,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
6,fbc61a1465db458e8b3cd321a,residential,ZAPOPAN,MEX,NaN,"[-103.4146957397461, 20.72429656982422]",Santa Margarita,45140,JALISCO,1266e1ef-f24f-11eb-82ac-169197452e1f,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
7,2f20645cb5cd4b3b89c2801aa,residential,SOLIDARIDAD,MEX,NaN,"[-87.07501220703125, 20.622051239013672]",Ejidal,77712,QUINTANA ROO,2f2df389-d61f-11ec-835d-162bfe278319,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
8,6fa561872264413a9acf8dbdb,residential,HERMOSILLO,MEX,NaN,"[-118.24368286132812, 34.0522346496582]",Insurgentes,83106,SONORA,5dcb1bb6-d07a-11ec-835d-0e35bba2c857,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55
9,98b9a964dffd4eaab07810671,residential,MONTERREY,MEX,NaN,"[-100.28665924072266, 25.66676139831543]",Buenos Aires,64800,NUEVO LEÓN,e315b767-7152-11eb-82ac-02fd26639ecf,...,2022-06-28 01:00:00,NaN,NaN,80ec8b10-25b7-48de-b425-7b93554002cb,2022-06-28 01:00:00,[],[],NaN,2022-06-29 16:49:55,2022-06-29 16:49:55


In [25]:
customerAddress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221470 entries, 0 to 221469
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   addressName         221470 non-null  object 
 1   addressType         221470 non-null  object 
 2   city                221470 non-null  object 
 3   country             221470 non-null  object 
 4   countryfake         0 non-null       float64
 5   geoCoordinate       221423 non-null  object 
 6   neighborhood        217181 non-null  object 
 7   postalCode          221470 non-null  object 
 8   state               221470 non-null  object 
 9   userId              221470 non-null  object 
 10  id                  221470 non-null  object 
 11  accountId           221470 non-null  object 
 12  accountName         221470 non-null  object 
 13  dataEntityId        221470 non-null  object 
 14  createdBy           221470 non-null  object 
 15  createdIn           221470 non-nul

In [26]:
customerAddress.describe()

,countryfake,auto_filter
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


In [28]:
customerAddress.isnull().sum()

addressName                0
addressType                0
city                       0
country                    0
countryfake           221470
geoCoordinate             47
neighborhood            4289
postalCode                 0
state                      0
userId                     0
id                         0
accountId                  0
accountName                0
dataEntityId               0
createdBy                  0
createdIn                  0
updatedBy             218650
updatedIn             218650
lastInteractionBy          0
lastInteractionIn          0
followers                  0
tags                       0
auto_filter           221470
Cretaed_Timestamp          0
Updaqted_Timestamp         0
dtype: int64

In [30]:
customerAddress.duplicated().sum()

np.int64(33)

In [31]:
customerAddress.nunique()


addressName           56229
addressType               3
city                   2717
country                   6
countryfake               0
geoCoordinate          6592
neighborhood          20398
postalCode            10759
state                   163
userId                50153
id                    56233
accountId                 9
accountName              13
dataEntityId              1
createdBy               391
createdIn               685
updatedBy               206
updatedIn               511
lastInteractionBy       398
lastInteractionIn       685
followers                 1
tags                      1
auto_filter               0
Cretaed_Timestamp      8669
Updaqted_Timestamp     8669
dtype: int64

## General Order Detail

In [27]:
generalOrder.head(10)

,Country,orderId,sequence,marketplaceOrderId,sellerOrderId,origin,affiliateId,salesChannel,merchantName,status,...,ClientId,RequestedByUser,RequestedBySystem,RequestedBySellerNotification,RequestedByPaymentNotification,RequestedByReason,cancellationData.Date,Cretaed_Timestamp,Updaqted_Timestamp,SourceSite
0,CO,MLB-41370001211,850110346,41370001211,NaN,Fulfillment,MLB,4,NaN,handling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
1,MX,LKT-v33516186ekt-01,520003,v33516186ekt-01,NaN,Fulfillment,LKT,4,NaN,invoiced,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-24 11:13:36,2021-11-24 11:13:36,ACR MX
2,CO,MLB-41378800118,850110330,41378800118,NaN,Fulfillment,MLB,4,NaN,handling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
3,MX,1167881777015-01,510055,NaN,00-1167881777015-01,Marketplace,NaN,3,NaN,invoiced,...,a68e2d84-7781-467f-8c2b-05f570060001,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-24 11:13:36,2021-11-24 11:13:36,ACR MX
4,CO,v850110335wcol-01,850110335,NaN,00-v850110335wcol-01,Marketplace,NaN,1,NaN,canceled,...,26fb40af-2a67-450f-8b6d-dbdb4e1880fb,True,NaN,NaN,NaN,No se aprobó el pago así que volveré a comprar,2022-05-18 01:00:00,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
5,CO,v850110320wcol-01,850110320,NaN,00-v850110320wcol-01,Marketplace,NaN,1,NaN,handling,...,e56a651e-03ae-4120-a1fd-d41cc1e2df30,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
6,CO,v850110327wcol-01,850110327,NaN,00-v850110327wcol-01,Marketplace,NaN,1,NaN,handling,...,dc6446e0-12ea-44a4-b370-5f67aa10708b,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
7,CO,MLB-41380176754,850110322,41380176754,NaN,Fulfillment,MLB,4,NaN,handling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
8,CO,v850110320wcol-01,850110320,NaN,00-v850110320wcol-01,Marketplace,NaN,1,NaN,handling,...,0e8068b0-7d14-40e5-a548-7f85eb36f316,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-10 00:40:13,2022-05-10 00:40:13,WHR CO
9,CO,v805106741kaco-01,805106741,NaN,00-v805106741kaco-01,Marketplace,NaN,1,NaN,handling,...,4a6566f0-0aa5-4f6b-8f70-25146200fe02,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-12 00:48:53,2022-06-12 00:48:53,KAID CO


In [32]:
generalOrder.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67934 entries, 0 to 67933
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         67934 non-null  object 
 1   orderId                         67934 non-null  object 
 2   sequence                        67934 non-null  int64  
 3   marketplaceOrderId              4950 non-null   object 
 4   sellerOrderId                   62984 non-null  object 
 5   origin                          67934 non-null  object 
 6   affiliateId                     4950 non-null   object 
 7   salesChannel                    67934 non-null  int64  
 8   merchantName                    1 non-null      object 
 9   status                          67933 non-null  object 
 10  statusDescription               67927 non-null  object 
 11  value                           67934 non-null  float64
 12  creationDate                    

In [33]:
generalOrder.describe()

,sequence,salesChannel,value,giftRegistryData,lastMessage,changesAttachment,roundingError,commercialConditionData,subscriptionData,taxData,checkedInPickupPointId
count,6.793400e+04,67934.000000,6.793400e+04,0.0,0.0,0.0,67934.0,0.0,0.0,0.0,0.0
mean,3.096779e+08,1.856213,3.586325e+07,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
std,4.032522e+08,1.253834,7.890474e+07,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
min,1.300000e+04,1.000000,0.000000e+00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
25%,5.461115e+05,1.000000,2.500000e+05,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
50%,6.274200e+05,1.000000,7.777780e+05,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
75%,8.051153e+08,3.000000,3.703704e+07,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
max,8.501232e+08,8.000000,1.439506e+09,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [34]:
generalOrder.duplicated().sum()

np.int64(0)

In [35]:
generalOrder.isnull().sum()


Country                               0
orderId                               0
sequence                              0
marketplaceOrderId                62984
sellerOrderId                      4950
origin                                0
affiliateId                       62984
salesChannel                          0
merchantName                      67933
status                                1
statusDescription                     7
value                                 0
creationDate                          0
lastChange                            0
orderGroup                         4950
marketplaceItems                      0
giftRegistryData                  67934
ratesAndBenefitsData              40100
followUpEmail                         0
lastMessage                       67934
hostname                              0
changesAttachment                 67934
openTextField                     64168
roundingError                         0
orderFormId                        4950


In [36]:
generalOrder.dtypes


Country                            object
orderId                            object
sequence                            int64
marketplaceOrderId                 object
sellerOrderId                      object
origin                             object
affiliateId                        object
salesChannel                        int64
merchantName                       object
status                             object
statusDescription                  object
value                             float64
creationDate                       object
lastChange                         object
orderGroup                         object
marketplaceItems                   object
giftRegistryData                  float64
ratesAndBenefitsData               object
followUpEmail                      object
lastMessage                       float64
hostname                           object
changesAttachment                 float64
openTextField                      object
roundingError                     

In [37]:
generalOrder.nunique()


Country                               6
orderId                           59310
sequence                          51405
marketplaceOrderId                 4832
sellerOrderId                     54450
origin                                2
affiliateId                           9
salesChannel                          7
merchantName                          1
status                               14
statusDescription                    14
value                             10276
creationDate                        630
lastChange                          630
orderGroup                        40810
marketplaceItems                      1
giftRegistryData                      0
ratesAndBenefitsData                520
followUpEmail                     56618
lastMessage                           0
hostname                             15
changesAttachment                     0
openTextField                       414
roundingError                         1
orderFormId                       55585


etso es una prueba 2